In [1]:
import pandas as pd
import syft as sy
import numpy as np

sy.logger.remove()

### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into italy... done!


In [3]:
# Search for the Dataset in canada domain
ca_domain_node.store.pandas[ca_domain_node.store.pandas["object_type"] == "<class 'syft.core.tensor.tensor.Tensor'>"]

,ID,Tags,Description,object_type
91,<UID: 2ea270613b3847b5a65ebc0a8b6fb98e>,[#40k-feb2020-numpy],,<class 'syft.core.tensor.tensor.Tensor'>


In [4]:
# Search for the Dataset in italy domain
it_domain_node.store.pandas[it_domain_node.store.pandas['object_type'] == "<class 'syft.core.tensor.tensor.Tensor'>"]

,ID,Tags,Description,object_type
151,<UID: 6e6878426c7b4845b8f8473941124386>,[#Italy-Numpy-feb2020-Tensor],,<class 'syft.core.tensor.tensor.Tensor'>


In [5]:
# Select the dataset pointers using the index

ca_dataset_ptr = ca_domain_node.store[91]
it_dataset_ptr = it_domain_node.store[151]

In [6]:
# Verfiy the dataset tensors via tags

ca_dataset_ptr.tags, it_dataset_ptr.tags

(['#40k-feb2020-numpy'], ['#Italy-Numpy-feb2020-Tensor'])

In [7]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

In [8]:
parties = [ca_domain_node, it_domain_node]

In [9]:
# Converting ca_dataset_ptr to mpc tensor

mpc_tensor = MPCTensor(secret=ca_dataset_ptr, shape=(40000, 3), parties=parties)

In [30]:
# Performing a Private / Public Addition

# Creating an array of ones of size (40000, 3)
# Converting float to int, for SMPC to work
e = np.ones((40000, 3))
e = e.astype(int)

result = mpc_tensor + e

In [38]:
result.reconstruct()

AuthorizationException: You do not have permission to .get() Object with ID: <UID: e9ad3fc70dead973eacd837212196e3f>Please submit a request.